In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import wandb
from pytorch_lightning.loggers import WandbLogger
import torch
import pytorch_lightning as pl
from Models.Gemformer import Gemformer
import torch.nn as nn
import torch.optim as optim
from ocpmodels.datasets import LmdbDataset
from torch.utils.data import random_split
import torch_geometric.loader as geom_loader
import torch_geometric.data as data
from pytorch_lightning.callbacks import LearningRateMonitor,ModelCheckpoint
from typing import Any
from pytorch_lightning.utilities.types import STEP_OUTPUT

c:\Users\lhuang37\.conda\envs\ocp-models\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import yaml
import torch.nn.init as init

# Model initialize

In [1]:



# Save the model hyperparameters to a YAML file
model_hparams = {
    "num_heads": 1,
    "emb_size_in": 256,
    "emb_size_trans": 64
}

with open('params/model_hparams.yml', 'w') as file:
    yaml.dump(model_hparams, file)

In [13]:
with open('params/model_hparams.yml', 'r') as file:
    loaded_model_hparams = yaml.load(file, Loader=yaml.FullLoader)

# Create the model using the loaded hyperparameters
model = Gemformer(**loaded_model_hparams)

# Apply Kaiming initialization to the model parameters
# for name, param in model.named_parameters():
#     if 'weight' in name:
#         print(param.shape)
        
#         init.kaiming_normal_(param, mode='fan_in', nonlinearity='relu')
#     elif 'bias' in name:
#         init.zeros_(param)


In [ ]:
CHECKPOINT_PATH="./checkpoints"
DEVICE='cuda'
dataset=LmdbDataset({"src":"Data/eoh_t.lmdb"})

train_length = int(0.8 * len(dataset))
val_length = len(dataset) - train_length

# Split the dataset into train and validation
train_dataset, val_dataset =random_split(dataset, [train_length, val_length])
# train_dataset=ReverseDataset(train_dataset,train_length)
# val_dataset=ReverseDataset(val_dataset,val_length)
train_loader = geom_loader.DataLoader(train_dataset, batch_size=1)
val_loader = geom_loader.DataLoader(val_dataset, batch_size=1)
wandb.init()
wandb_logger = WandbLogger()

In [ ]:
from tqdm import tqdm

# Unfinished

In [ ]:
def train_fn(data_loader,model,optimizer):
    model.train()
    total_loss=0.0
    

    for images,masks in tqdm(data_loader):
        images=images.to(DEVICE)
        masks=masks.to(DEVICE)


        optimizer.zero_grad()
        preds=model(images)
        loss=
        loss.backward()
        optimizer.step()

        total_loss+=loss.item()
        

    return total_loss/len(data_loader)

In [ ]:
def eval_fn(data_loader,model):

    model.eval()
    total_loss=0.0
    total_acc=0
    
    with torch.no_grad():

        for images,masks in tqdm(data_loader):
            images=images.to(DEVICE)
            masks=masks.to(DEVICE)        
            logits,loss=model(images,masks)

            total_loss+=loss.item() 

            mask=torch.sigmoid(logits[0]).cpu().squeeze().flatten()        
            
            mask_true=masks[0].cpu().squeeze(0).flatten()
            
            
            precision=roc_auc_score(mask_true,mask)
            total_acc+=precision
                     
          
            

    return total_loss/len(data_loader),total_acc/len(data_loader)

In [ ]:
optimizer=torch.optim.Adam(model.parameters(),lr=0.005)
best_valid_loss=np.Inf
animator=d2l.Animator(xlabel='epoch',ylabel='loss',yscale='log',xlim=[1,epochs],ylim=[0.1,1.5],legend=['train','valid','acc'])

for i in range(epochs):
    train_loss=train_fn(trainloader,model,optimizer)
    valid_loss,acc=eval_fn(validloader,model)
    print(acc)
    animator.add(i+1,(train_loss,valid_loss,acc))
    if valid_loss< best_valid_loss:
        torch.save(model.state_dict(),'best_model_n.pt')
        print('saved-model')
        best_valid_loss=valid_loss

    print(f'epoch:{i+1} Train_loss:{train_loss} Valid_loss:{valid_loss} Valid acc:{acc}')